In [20]:
from azure.ai.ml import command, Input
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

In [21]:
sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

mc_client = MLClient(DefaultAzureCredential(), sub_id, rg, workspace)
print(mc_client)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001F56E9F6C20>,
         subscription_id=b746917e-ceb7-4ae0-81e6-3ccd893cb0de,
         resource_group_name=dpv2,
         workspace_name=dpv2-wks)


In [22]:
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Data
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes


In [23]:
inputs = {
    "cifar_zip": Input(
        path="https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz",
        type=AssetTypes.URI_FILE,
    )
}

# metang:try: how about we try the uri_file here. Seems not ok, as the code is to extract the gz files. We dont know how many files in the .gz
outputs = {
    "cifar": Output(
        # write to absolute path in the default datastore        
         path=f"azureml://subscriptions/{sub_id}/resourcegroups/{rg}/workspaces/{workspace}/datastores/workspaceblobstore/paths/CIFAR-10",
         type=AssetTypes.URI_FOLDER
    )
}

In [24]:
from calendar import c


job = command(
    code = "./src",
    # metang:note: when I mistype mc_clinet(DefaultCredential() ,... ) to mc_clinet(DefaultCredential, "missing ()" ...),
    # got error message: AssetException: Error with code: 'str' object has no attribute '_successful_credential'    
    command="python read_write_data.py --input_data ${{inputs.cifar_zip}} --output_folder ${{outputs.cifar}}",
    inputs=inputs, 
    outputs=outputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:9",
    compute="cpu-cluster",
)

In [43]:
ret_job = mc_client.jobs.create_or_update(job, experiment_name="distributed_job")


In [ ]:
ret_job.services["Studio"].endpoint

In [38]:
print(ret_job.name)
print(ret_job.experiment_name)
print(ret_job.outputs["cifar"])
# metang:?: sample doesn't work for the following code,
#print(ret_job.outputs["cifar"]["path"])

polite_brain_h1g2cp710x
distributed_job
${{parent.jobs.polite_brain_h1g2cp710x.outputs.cifar}}


In [19]:
# Test Credential

try: 
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
    print("credential ok")
except Exception as ex:
    print(ex)

credential ok


In [40]:
for i in mc_client.jobs.list():
    if i.name == ret_job.name:
        print(i.status)

Running


In [47]:
inputs = dict(
    cifar = Input(
        type=AssetTypes.URI_FOLDER,
        #path = ret_job.outputs["cifar"]["path"],        
        path=f"azureml://subscriptions/{sub_id}/resourcegroups/{rg}/workspaces/{workspace}/datastores/workspaceblobstore/paths/CIFAR-10", 
    ),
    epoch=10,
    batchsize=64,
    workers=2,
    lr=0.01,
    momen=0.9,
    prtfreq=200,
    outputs="./outputs",
)

In [ ]:
dist_job = command(
    code = "./src",
    command="python train.py --data-dir ${{inputs.cifar}} --epochs ${{inputs.epoch}} --batch-size ${{inputs.batchsize}} --workers ${{inputs.workers}} --learning-rate ${{inputs.lr}} --momentum ${{inputs.momen}} --print-freq ${{inputs.prtfreq}} --model-dir ${{inputs.output}}",
    inputs=inputs,
    environment="azureml:AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6",
    compute="gpu-cluster-V100",  # Change the name to the gpu cluster of your workspace.
    instance_count=2,  # In this, only 2 node cluster was created.
    distribution={
        "type": "PyTorch",
        # set process count to the number of gpus per node
        # NV6 has only 1 GPU
        "process_count_per_instance": 1,
    },
)